In [ ]:
%load_ext lab_black
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile

sns.set()
from skimage.measure import regionprops, regionprops_table

names = {0: "junk", 1: "dapi", 2: "gfap", 3: "neun", 4: "olig2", 5: "tmem119"}


def setup():
    imgs = tifffile.imread("E:\\Lab\\V10B01-087_C1.tif")
    c, y, x = imgs.shape
    target = imgs[:, y // 4 : y // 2, x // 4 : x // 2]
    masks = np.load("masks.npy")
    its = {
        names[i]: regionprops_table(
            masks, intensity_image=target[i], properties=["intensity_mean"]
        )["intensity_mean"]
        for i in range(2, 6)
    }

    general = regionprops_table(masks, properties=["centroid", "area"])
    its["area"] = general["area"]
    its["y"] = general["centroid-0"]
    its["x"] = general["centroid-1"]

    return pd.DataFrame(its), masks, target


df, masks, target = setup()

props = regionprops(masks)

In [ ]:
df += 4 * np.random.normal(size=df.shape)

In [ ]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

brush = alt.selection_interval(name="brush")

alt.Chart(df, height=150, width=150).mark_circle(size=3, opacity=0.1).encode(
    x=alt.X(alt.repeat("column"), type="quantitative", scale=alt.Scale(type="linear")),
    y=alt.Y(alt.repeat("row"), type="quantitative", scale=alt.Scale(type="linear")),
    color=alt.condition(brush, alt.ColorValue("#084c96"), alt.ColorValue("lightgray")),
).repeat(
    row=["gfap", "neun", "olig2", "tmem119", "area"],
    column=["gfap", "neun", "olig2", "tmem119", "area"],
).add_selection(
    brush
)

In [ ]:
import panel as pn
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set()
pn.extension("vega")

brush = alt.selection_interval(name="brush")


def gen_chart(name: str):
    return (
        alt.Chart(df, height=150)
        .mark_circle(opacity=0.2, size=4)
        .encode(
            x="area",
            y=name,
            color=alt.condition(
                brush, alt.ColorValue("#084c96"), alt.ColorValue("lightgray")
            ),
        )
        .properties(width=250, height=250)
        .add_selection(brush)
    )


chart = gen_chart("neun")
chart &= gen_chart("tmem119")
chart &= gen_chart("gfap")
chart &= gen_chart("olig2")

vega_pane = pn.pane.Vega(chart, debounce=10)


def filtered_table(selection):
    if not selection:
        return None
    query = " & ".join(
        f"{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}"
        for col, crange in selection.items()
    )
    idxs = df.query(query).index
    if not len(idxs):
        return None
    if len(idxs) == 1:
        return pn.pane.Matplotlib(pl(idxs))
    return pn.pane.Matplotlib(pl(idxs[:3]))


def pl(idx: list[int]):
    pad = 10
    fig, axs = plt.subplots(nrows=3, ncols=6, figsize=(12, 6), dpi=120)
    axs = axs.flatten()
    for ax in axs:
        ax.clear()

    for j in range(min(3, len(idx))):
        bbox = props[idx[j]]["bbox"]
        roi = props[idx[j]]["image"]
        axs[6 * j + 0].imshow(
            np.pad(roi, (pad, pad), mode="constant", constant_values=0), aspect="equal"
        )
        axs[6 * j + 0].set_title("Mask")
        axs[6 * j + 0].grid(False)
        for i in range(1, 6):
            im = axs[6 * j + i].imshow(
                target[
                    i,
                    max(0, bbox[0] - pad) : min(target.shape[1], bbox[2] + pad),
                    max(0, bbox[1] - pad) : min(target.shape[2], bbox[3] + pad),
                ],
                vmax=200,
                aspect="equal",
            )
            axs[6 * j + i].set_title(names[i])
            axs[6 * j + i].grid(False)
    # fig.colorbar(im)
    fig.tight_layout()
    return fig


pn.Row(vega_pane, pn.bind(filtered_table, vega_pane.selection.param.brush))

In [ ]:
corr = df[["gfap", "neun", "olig2", "tmem119", "area"]].corr()
corr.style.background_gradient(cmap="Spectral")